In [1]:
import os
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from IPython import display
from IPython.display import HTML
from tempfile import NamedTemporaryFile
from netCDF4 import Dataset

In [2]:
port = os.environ.get("APP_PORT", 5000)
base_url = f"http://localhost:{port}"

In [3]:
input_url = base_url + f"/osprey/input?case_id=sample&run_startdate=2015-12-01-00&stop_date=2015-12-31&lons=-116.46875&lats=50.90625&names=BCHSP"
input_url

'http://localhost:5002/osprey/input?case_id=sample&run_startdate=2015-12-01-00&stop_date=2015-12-31&lons=-116.46875&lats=50.90625&names=BCHSP'

In [4]:
input_response = requests.get(input_url)
status_url = base_url + input_response.content.split()[-1].decode("utf-8")
status_url

'http://localhost:5002/osprey/status/f1e9c34f-1c2c-41a3-ab2c-a35a00a7d163'

In [5]:
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

In [6]:
driver.get(status_url)

In [7]:
while True:
    try:
        driver.find_element(By.CLASS_NAME, "progress-bar-header").text
        display.display(HTML(driver.page_source))
        display.clear_output(wait=True)
        time.sleep(0.5)
    except NoSuchElementException:
        break
display.display(HTML(driver.page_source))

In [8]:
completed_text = driver.find_element(By.TAG_NAME, "body").text
output_url = base_url + completed_text.split()[-1]
output_url

'http://localhost:5002/osprey/output/f1e9c34f-1c2c-41a3-ab2c-a35a00a7d163'

In [9]:
output_response = requests.get(output_url)
with NamedTemporaryFile(dir = os.environ.get("TMPDIR", "/tmp")) as fp:
    fp.write(output_response.content)
    data = Dataset(fp.name)

In [10]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: RVIC history file
    comment: Output from the RVIC Streamflow Routing Model.
    Conventions: CF-1.6
    history: Created: Wed Mar  1 15:29:23 2023
    source: /home/eyvorchuk/code/osprey/osprey-venv/bin/osprey
    institution: University of Washington
    hostname: pcic-2024
    username: eyvorchuk
    casename: sample
    casestr: historical
    references: Based on the initial model of Lohmann, et al., 1996, Tellus, 48(A), 708-721
    version: 1.1.6
    RvicPourPointsFile: tmplw_9cfha.csv
    RvicUHFile: tmp_hvn6w7x.csv
    RvicFdrFile: pcic.pnw.rvic.input_20170927.nc
    RvicDomainFile: domain.pnw.pcic.20170927.nc
    featureType: timeSeries
    dimensions(sizes): time(31), nv(2), outlets(1), nc_chars(256)
    variables(dimensions): float32 time(time), float32 time_bnds(time, nv), float32 lon(outlets), float32 lat(outlets), int32 outlet_x_ind(outlets), int32 outlet_y_ind(outlets), int3

In [11]:
driver.quit()